# Information Retrieval project 2022

## 🧑‍🎓 Names
+ Mashra Marwan
+ Kiselov Nikita
+ Serré Gaëtan

## Imports

In [2]:
import sys
import time
import json
import pprint
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span
from collections import Counter
from nltk.tokenize import MWETokenizer
from nltk.util import Trie
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
tqdm.pandas()
spacy.__version__ 



Our first goal here is to create a model capable of recognizing the term in our patents. For that, we will be using a Named Entity Recognition model given by `spacy`.

## 🗃️ Read and preprocess data

In [2]:
# if you've already unzipped the file
patent_data=open('G06K.txt').read().strip()

# split into patents texts | 1 entry = 1 patent
patent_texts = patent_data.split('\n\n')

# split each patent into lines
patent_lines = patent_data.split('\n')

## 👀 Extract features

In [3]:
# here are the potential terms
mwes = open('manyterms.lower.txt').read().lower().strip().split('\n')
print(mwes[44444:44456])
print(len(mwes),'mwes')

['antonio superchi', 'antonio tarver', 'antonio torres jurado', 'antonio valdes', 'antonio valdes y fernandez bazan', 'antonio valdez', 'antonio valdés y bazán', 'antonio valdés y fernández bazán', 'antonio valente', 'antonio vitali', 'antonio vivaldi', 'antonio xavier machado e cerveira']
743274 mwes


### We extract the terms from our patents using manyterms

In [4]:
# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(1, 4), stop_words="english", vocabulary=mwes, lowercase=True)
X=cvectorizer.fit_transform(patent_texts)

# Show top-25 most frequent terms
termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names()).T.sort_values(by = 0, ascending = False)
termdf_cv.head(25)

/Users/gaetanserre/miniconda3/envs/NLP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
/Users/gaetanserre/miniconda3/envs/NLP/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0
electronic device,16280
image processing,12224
control unit,9263
mobile terminal,9165
information processing,7732
neural network,6734
user interface,6177
computer readable,6103
fingerprint sensor,5980
display device,5666


## 🪄 SpaCy NER

Let's start from understanding. Here is an example of showing part of text on one patent with default NER.

In [5]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(patent_texts[0][18000:20000]) # 
displacy.render(doc, style="ent", jupyter = True)

We want to create a such model capable of recognizing the terms that are in the context of our patents. For that, we need to create a dataset and we will be using `manyterms` as a terms database.

### Create DataSet

We need to create propper dataset that is compatible with SpaCy 3.0 to train a NER model.

In [6]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in termdf_cv.index]
matcher.add("Tech", patterns)

In [7]:
train_lines, test_lines = train_test_split(patent_lines, test_size=0.3, random_state=42)

We are using PharsesMatcher to find entities similar to one from mayterms.txt  
Then Span is labeled and saved into the binary `.spacy` format

In [8]:
def create_dataset(text, n_lines, filename, offset=0):
  LABEL = "TECH"
  doc_bin = DocBin() # create a DocBin object

  for training_example  in tqdm(text[offset:offset+n_lines]):
      doc = nlp.make_doc(training_example) 
      ents = []
      
      for match_id, start, end in matcher(doc):
          span = Span(doc, start, end, label=LABEL)
          if span is None:
              print("Skipping entity")
          else:
              ents.append(span)

      filtered_ents = filter_spans(ents)
      doc.ents = filtered_ents 
      doc_bin.add(doc)
  doc_bin.to_disk(filename)

In [9]:
create_dataset(train_lines, 40_000, "training_data.spacy")
create_dataset(test_lines, 12_000, "valid_data.spacy")

  0%|          | 0/40000 [00:00<?, ?it/s]

  0%|          | 0/12000 [00:00<?, ?it/s]

Now that our datasets are created, we can train a spacy NER model.

### Train the model

Donwnload __base_config.cfg__ for your system at https://spacy.io/usage/training#quickstart

In [10]:
# Run to generate full training config
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Run the training. The best and last model will be stored into __./spacy_output__  

In [11]:
# !python -m spacy train config.cfg --output ./spacy_output --paths.train ./training_data.spacy --paths.dev ./valid_data.spacy --gpu-id 0

### Test the model

In [12]:
nlp = spacy.load("spacy_output/model-best")

doc = nlp("Wi-Fi Direct (registered trademark, which will be hereinafter referred to as WFD) \
           corresponding to a technology for directly performing a communication based on a \
           wireless LAN between communication devices without intermediation of an access \
           point (hereinafter referred to as AP) is standardized in Wi-Fi Alliance serving \
           as a wireless LAN industry group.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

/Users/gaetanserre/miniconda3/envs/NLP/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In our patents, a lot of terms are abbreviated.
For example, `WI-FI Direct` is mentioned as `WFD`, or `P2P Group Owner` as `GO`.

Our original model does not recognize these abbreviations and therefore a huge part of the terms are ignored.
So we will fine-tune our model using prodigy

## 🦄 Prodigy

We create a train dataset for fine-tuning our ner model.

In [ ]:
# !prodigy ner.correct fine_tune_g06k2 spacy_output/model-best G06K.txt --loader txt --label TECH

<img src=img/annotations.png style="width: 40%; height:40%"></img>

Now let's fine-tune our model!

In [ ]:
# !prodigy train ./prodigy_output/ --ner fine_tune_g06k --base-model spacy_output/model-best --gpu-id 0

Now our refined model should recognize better the abbreviations.

In [13]:
nlp = spacy.load("prodigy_output/model-best")

doc = nlp("According to the WFD, the communication is performed when one of the \
           communication devices that directly perform the wireless LAN communication \
           operates as the AP. According to the WFD, a role of the device that operates \
           as the AP will be referred to as P2P Group Owner (hereinafter, referred to as GO). \
           On the other hand, a role of the device that participates in a network generated by \
           the GO will be referred to as P2P Client (hereinafter, referred to as CL). \
           According to the WFD, a communication parameter necessary for participating in \
           the network generated by the GO is shared between the devices by transmitting the \
           communication parameter from the GO to the CL, and thereafter, the wireless \
           communication according to the WFD is executed on the basis of the shared communication parameter.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

This fine tuned model is more accurate than the first one. We will use it in our Hearst patterns recognition.

## 🧬 Hearst Patterns

In [ ]:
class Hearst_Patterns:
    """ Extracts hearst patterns from a corpus
    """
    def __init__(self, patterns_file="patterns.json", model_path="spacy/model-new", text_path="G06K.txt"):
        """ creates an instance of the class Hearst_Patterns

        Args:
            patterns_file (path, optional): the json file containing the patterns. Defaults to "patterns.json".
            model_path (path, optional): the folder containing the NER model to use. Defaults to "spacy/model-new".
            text_path (path, optional): the file containing the corpus analyse and extract patterns. Defaults to "G06K.txt".
        """

        # read the text file
        g06k = open(text_path).read().strip()
        self.patent_lines = g06k.split('\n')
        
        # load the models
        self.nlp = spacy.load(model_path)
        self.en_nlp = spacy.load("en_core_web_lg")
        self.nlp.add_pipe("merge_entities")
        self.en_nlp.add_pipe('merge_noun_chunks')

        self.matcher = Matcher(self.nlp.vocab)
        self.patterns = self.load_patterns_from_json(patterns_file)
        for name, pattern in self.patterns:
            self.matcher.add(name, pattern)

        # this list is used in the method get_matches
        self.continue_words = [',','and','or',';','also','as well']


    def load_patterns_from_json(self, patterns_file):
        """ read the json file and return the list of

        Args:
            patterns_file (path): the json file containing the patterns

        Returns:
            List: a list of the hearst patterns found in the json file  
        """
        f = open(patterns_file)
        data = json.load(f)
        patterns = []
        for name, pattern in data.items():
            patterns.append((name, pattern))

        return patterns


    def extract_patterns(self, size=10, save_folder=".", start=0):
        """ look for matches in a corpus (text file)

        Args:
            size (int, optional): the minimum number of matches to be found. Defaults to 10.
            save_folder (path, optional): the folder in which save the resulted csv file. Defaults to ".".
            start (int, optional): the first line in which we start to look for matches (useful to continue where you stopped). Defaults to 0.
        """
        extraced_patterns = []

        # chose a start
        line = start
        count = 0

        # for the output 
        print(f'{count} pattern extracted...', end='\r')
        sys.stdout.flush()

        while count<size:
            while True: # we read line by line until finding a match, to keep track of the count 
                try: # it bugs very rarely, don't know why XD  

                    # look for a match
                    patterns = self.get_matches(self.patent_lines[line])
                    if patterns:
                        extraced_patterns += patterns
                        break
                    print(f'{count} patterns extracted...{line}', end='\r')
                    sys.stdout.flush()
                except:
                    print("An error has occurred")
                
                line += 1
            count = len(extraced_patterns)
            print(f'{count} patterns extracted...{line}', end='\r')
            sys.stdout.flush()
            

        print(f'({count}) patterns extracted from lines ({start}-{line}))')
        save_file = f"{save_folder}/hearst_patterns.{len(extraced_patterns)}.csv"
        print(f'Patterns saved to {save_file}')
        df = pd.DataFrame(extraced_patterns, columns =['word1', 'word2', 'relation', 'label', 'text'])
        df.to_csv(save_file)

    def get_matches(self, text):
        label = {
            'rhyper':-1,
            'hyper':1,
        }
        doc = self.nlp('. '+text) # because patterns like < !(bla bla) X > don't work when X is in the beginning of the sentence 
        
        matches = self.matcher(doc)
        relations = []
        for match_id, start, end in matches:

            # get all entities indices in the doc
            ent_indices = [i for i in range(start,end) if doc[i].text in [ent.text for ent in doc[start:end].ents]]
            if not ent_indices: # no entity found
                return []

            # extract X...Y from a match ..X...Y.., so now we know that the first and the last token are the entities
            span = doc[min(ent_indices):max(ent_indices)+1]

            match_info = self.nlp.vocab.strings[match_id]  # Get string representation
            match_name = match_info.split('-')[0]   # hyper or rhyper
            match_type = match_info.split('-')[1]   # single or multi

            np_0 = span[0]  # left term
            np_1 = span[-1] # right term (or first right term if multiple)

            # all the right terms (ex. for Y...X1, X2, ...Xn) X1...Xn are the right terms
            right_terms = [np_1.text]
            if match_type=="multi": # look for other terms (X2,X3..etc)

                # we use the en_core_web_lg model to get the noun chunks
                doc_en = self.en_nlp(doc[end:].text)
                for d in doc_en:
                    # look for entities inside the noun chunk
                    matching_ents = [ent.text for ent in doc.ents if ent.text in d.text]
                    if matching_ents:
                        right_terms.append(matching_ents[0])
                    elif d.text not in self.continue_words:  # stop when seeing a word that's not in the list
                        break

            for term in right_terms:
                relations.append((np_0.text, term, match_name, label[match_name], text))

        relations = set(relations)
        return list(relations)

### ⛷️ start the extraction

In [ ]:
hp = Hearst_Patterns(patterns_file="patterns.json", model_path="../spacy/model-new", text_path="../G06K.txt")
hp.extract_patterns(size=50, start=5896, save_folder="hearst_patterns/")

time: 0 ns (started: 2022-04-21 02:13:14 +02:00)


### ✨ Results

in the first <b>26331</b>, we found <b>245</b> different matches, <b>155</b> unique relation. Here are some of the results:

In [11]:
df_results = pd.read_csv("hearst_patterns/hearst_patterns.155.csv", index_col=0)
df_results[['Hypernym','Hyponym','Frequency']].head(10)

,Hypernym,Hyponym,Frequency
0,home appliance,mobile robot,15
1,computer network,the Internet,10
2,artificial intelligence,Deep learning,6
3,electronic device,electronic component,4
4,storage device,hard disk,4
5,communication network,the Internet,4
6,point light source,light source,4
7,the environment,foot traffic,3
8,storage media,hard drives,3
9,laser projector,display device,3


## 🏆 Evaluation

#### Load model and data

In [ ]:
# load best model
nlp_ner = spacy.load("./spacy_output/model-best")

# load test lines
with open('test_lines.txt', 'r') as f:
    test_lines = f.readlines()
f.close()

# read homonym_list.txt with pairs of homonyms
homonyms_df = pd.read_csv('hearst_patterns.30.csv')
homonyms_df

#### 🏅 Manual Gold dataset
Here we are evaluating NER model on the manually created gold dataset. 

In [ ]:
!TODO__TODO

#### 🌐 Word-Net

Here we are evaluating extracted Hypernyms using WordNet. Here is an example how it works

In [ ]:
cd_rom = wn.synsets('CD-ROM', pos='n')
computer = wn.synsets('computer', pos='n')

In [ ]:
for synset in cd_rom:
    for synset2 in computer:
        print(synset, synset2)
        print("Score:", synset.wup_similarity(synset2))

In [ ]:
cd_rom[0].shortest_path_distance(computer[0])

Run on our list of hypernyms

In [ ]:
def wordnet_distance(word1, word2):
    parent = wn.synsets(word1.replace(' ', '_'))
    subclass = wn.synsets(word2.replace(' ', '_'))
    scores = [0]
    try:
        for synset in parent:
            for synset2 in subclass:
                scores.append(synset.wup_similarity(synset2))
    except:
        return np.nan

    return np.round(max(scores), 2)

In [ ]:
# iterate throw homonyms_df
results_wordnet = []
for index, row in tqdm(homonyms_df.iterrows()):
    if row["label"] == -1:
        parent = row['word1']
        subclass = row['word2']
    else:
        parent = row['word2']
        subclass = row['word1']
    res = wordnet_distance(parent, subclass)
    results_wordnet.append(res)
    print(parent, "⬅️", subclass, ": ", res)


Save to the dataframe

In [ ]:
# save to dataframe
homonyms_df['wordnet_distance'] = results_wordnet 

#### 🧪 Spacy embeddings
What if model already has links between words? Since it's trained on the corpus data, it should be able to find similarity between words.  
This is what can be useful while evaluationg our hyponyms list. We could run it and find low-similar elemts for further analysis.

In [ ]:
# load large model for comparison
nlp = spacy.load("./spacy_output/model-best")

word_1 = nlp("cloud platform")
word_2 = nlp("service provider")

print(word_1, "<->", word_2, word_1.similarity(word_2))

Run on our list of hypernyms

In [ ]:
def spacy_score(word1, word2):
    word1 = nlp(word1)
    word2 = nlp(word2)
    return word1.similarity(word2)

In [ ]:
# iterate throw homonyms_df
results_spacy = []
for index, row in tqdm(homonyms_df.iterrows()):
    if row["label"] == -1:
        parent = row['word1']
        subclass = row['word2']
    else:
        parent = row['word2']
        subclass = row['word1']

    res = spacy_score(parent, subclass)
    results_spacy.append(res)
    print(parent, "⬅️", subclass, ": ", res)


Add results to the dataframe

In [ ]:
homonyms_df['spacy_distance'] = results_spacy

In [ ]:
homonyms_df

#### 📜 Wikidata

In [ ]:
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)
import wptools

In [ ]:
parent_name = "computer"
candidate_name = "iPad"

# get Wikidata item for parent
page = wptools.page(parent_name)
data = page.get_parse(show=False)
q_parent_class = data.data['wikibase']
q_parent_class

In [ ]:
# use convenience function to get subclasses of an item as a list of item ids
subclasses_list = get_subclasses_of_item(q_parent_class)
len(subclasses_list)

In [ ]:
#print some of this classes
for subclass in subclasses_list[:5]:
    q42_dict = get_entity_dict_from_api(subclass)
    print(WikidataItem(q42_dict).get_label())

In [ ]:
page = wptools.page(candidate_name)
data = page.get_parse(show=False)
data.data['wikibase']

In [ ]:
print(f"Is class `{parent_name}` is a subclass of `{candidate_name}`: ", data.data['wikibase'] in subclasses_list)

Run on our list of hypernyms

In [ ]:
def wikidata_is_subclass(word_1, word_2):
    parent_name = word_1
    candidate_name = word_2

    # get Wikidata item for parent
    page = wptools.page(parent_name)
    try:
        data_parent = page.get_parse(show=False)
    except:
        print(f"Could not find Wikidata item for `{parent_name}`")
        return "⚠️"
    q_parent_id = data_parent.data['wikibase']

    # get Wikidata item for candidate
    page = wptools.page(candidate_name)
    try:
        data_subclass = page.get_parse(show=False)
    except:
        print(f"Could not find Wikidata item for `{candidate_name}`")
        return "⚠️"
    q_subclass_id = data_subclass.data['wikibase']

    # use convenience function to get subclasses of an item as a list of item ids
    subclasses_list = get_subclasses_of_item(q_parent_id)

    res = q_subclass_id in subclasses_list
    if res:
        return "✅"
    else:
        return "❌"

In [ ]:
# iterate throw homonyms_df
wikidata_results = []
for index, row in tqdm(homonyms_df.iterrows()):
    if row["label"] == -1:
        parent = row['word1']
        subclass = row['word2']
    else:
        parent = row['word2']
        subclass = row['word1']

    res = wikidata_is_subclass(parent, subclass)
    wikidata_results.append(res)
    print(parent, "⬅️", subclass, ": ",wikidata_is_subclass(parent, subclass))


Add results to the dataframe

In [ ]:
homonyms_df['wikidata_is_subclass'] = wikidata_results

In [ ]:
homonyms_df

#### Process table

In [ ]:
# create columns wordnet_is_subclass, fill with "⚠️" if not 0 or np.nan, otherwise fill with "✅" if wordnet_distance > 0.7, otherwise fill with "❌"
homonyms_df['wordnet_is_subclass'] = homonyms_df['wordnet_distance'].apply(lambda x: "⚠️" if x == 0 or np.isnan(x) else "✅" if x > 0.7 else "❌")

# do the same for spacy
homonyms_df['spacy_is_subclass'] = homonyms_df['spacy_distance'].apply(lambda x: "⚠️" if x == 0 or np.isnan(x) else "✅" if x > 0.3 else "❌")


homonyms_df

In [ ]:
homonyms_df[[ "label", "word1", "word2", "wordnet_is_subclass", "spacy_is_subclass", "wikidata_is_subclass"]].to_csv("./homonyms_results.csv", index=False)

In [ ]:
homonyms_df[["label", "word1", "word2",  "wordnet_distance", "spacy_distance", "wordnet_is_subclass", "spacy_is_subclass", "wikidata_is_subclass"]].to_csv("./homonyms_results_detailed.csv", index=False)